In [1]:
import cell2cell as c2c
import scanpy as sc
import pandas as pd

In [2]:
adata = sc.read_csv("./input/CID4465_count.csv").T
meta = pd.read_csv("./input/CID4465_metadata.csv", index_col = "Cell")
adata.obs = meta
adata.obs.head(10)

,Annotation,stim,Donor
Cell,,,
CID4465_AAACGGGTCATCGATG,Endothelial,CTRL,CID4465
CID4465_AAACGGGTCGGAAACG,Endothelial,CTRL,CID4465
CID4465_AAAGATGAGGTGTGGT,Endothelial,CTRL,CID4465
CID4465_AAAGCAATCCTTTACA,Endothelial,CTRL,CID4465
CID4465_AACCATGGTACGCTGC,Endothelial,STIM,CID4465
CID4465_AACGTTGGTACGAAAT,Endothelial,STIM,CID4465
CID4465_AACTCAGCATTGTGCA,Endothelial,STIM,CID4465
CID4465_AACTCCCGTAGCTGCC,Endothelial,CTRL,CID4465
CID4465_AACTGGTTCGGTGTCG,Endothelial,STIM,CID4465


In [3]:
lr_pairs = pd.read_csv("./input/Human-2020-Jin-LR-pairs.csv")
lr_pairs = lr_pairs.astype(str)
lr_pairs.head()

,interaction_name,pathway_name,ligand,receptor,agonist,antagonist,co_A_receptor,co_I_receptor,evidence,annotation,interaction_name_2,ligand_symbol,receptor_symbol,ligand_ensembl,receptor_ensembl,interaction_symbol,interaction_ensembl
0,TGFB1_TGFBR1_TGFBR2,TGFb,TGFB1,TGFbR1_R2,TGFb agonist,TGFb antagonist,nan,TGFb inhibition receptor,KEGG: hsa04350,Secreted Signaling,TGFB1 - (TGFBR1+TGFBR2),TGFB1,TGFBR1&TGFBR2,ENSG00000105329,ENSG00000106799&ENSG00000163513,TGFB1^TGFBR1&TGFBR2,ENSG00000105329^ENSG00000106799&ENSG00000163513
1,TGFB2_TGFBR1_TGFBR2,TGFb,TGFB2,TGFbR1_R2,TGFb agonist,TGFb antagonist,nan,TGFb inhibition receptor,KEGG: hsa04350,Secreted Signaling,TGFB2 - (TGFBR1+TGFBR2),TGFB2,TGFBR1&TGFBR2,ENSG00000092969,ENSG00000106799&ENSG00000163513,TGFB2^TGFBR1&TGFBR2,ENSG00000092969^ENSG00000106799&ENSG00000163513
2,TGFB3_TGFBR1_TGFBR2,TGFb,TGFB3,TGFbR1_R2,TGFb agonist,TGFb antagonist,nan,TGFb inhibition receptor,KEGG: hsa04350,Secreted Signaling,TGFB3 - (TGFBR1+TGFBR2),TGFB3,TGFBR1&TGFBR2,ENSG00000119699,ENSG00000106799&ENSG00000163513,TGFB3^TGFBR1&TGFBR2,ENSG00000119699^ENSG00000106799&ENSG00000163513
3,TGFB1_ACVR1B_TGFBR2,TGFb,TGFB1,ACVR1B_TGFbR2,TGFb agonist,TGFb antagonist,nan,TGFb inhibition receptor,PMID: 27449815,Secreted Signaling,TGFB1 - (ACVR1B+TGFBR2),TGFB1,ACVR1B&TGFBR2,ENSG00000105329,ENSG00000135503&ENSG00000163513,TGFB1^ACVR1B&TGFBR2,ENSG00000105329^ENSG00000135503&ENSG00000163513
4,TGFB1_ACVR1C_TGFBR2,TGFb,TGFB1,ACVR1C_TGFbR2,TGFb agonist,TGFb antagonist,nan,TGFb inhibition receptor,PMID: 27449815,Secreted Signaling,TGFB1 - (ACVR1C+TGFBR2),TGFB1,ACVR1C&TGFBR2,ENSG00000105329,ENSG00000123612&ENSG00000163513,TGFB1^ACVR1C&TGFBR2,ENSG00000105329^ENSG00000123612&ENSG00000163513


In [4]:
meta = adata.obs.copy()

In [5]:
meta.head()

,Annotation,stim,Donor
Cell,,,
CID4465_AAACGGGTCATCGATG,Endothelial,CTRL,CID4465
CID4465_AAACGGGTCGGAAACG,Endothelial,CTRL,CID4465
CID4465_AAAGATGAGGTGTGGT,Endothelial,CTRL,CID4465
CID4465_AAAGCAATCCTTTACA,Endothelial,CTRL,CID4465
CID4465_AACCATGGTACGCTGC,Endothelial,STIM,CID4465


In [6]:
interactions = c2c.analysis.SingleCellInteractions(rnaseq_data=adata.to_df().T,
                                                   ppi_data=lr_pairs,
                                                   metadata=meta,
                                                   interaction_columns=('ligand_symbol', 'receptor_symbol'),
                                                   communication_score='expression_thresholding',
                                                   expression_threshold=0.05, # values after aggregation
                                                   cci_score='bray_curtis',
                                                   cci_type='undirected',
                                                   aggregation_method='nn_cell_fraction',
                                                   barcode_col='Cell',
                                                   celltype_col='Annotation',
                                                   complex_sep='&',
                                                   verbose=False)

In [7]:
interactions.compute_pairwise_communication_scores()

Computing pairwise communication
Computing communication score between PVL and CAFs
Computing communication score between PVL and B-cells
Computing communication score between T-cells and CAFs
Computing communication score between Cancer Epithelial and Cancer Epithelial
Computing communication score between Endothelial and PVL
Computing communication score between T-cells and T-cells
Computing communication score between Myeloid and B-cells
Computing communication score between PVL and Cancer Epithelial
Computing communication score between PVL and T-cells
Computing communication score between PVL and Myeloid
Computing communication score between B-cells and Myeloid
Computing communication score between CAFs and Myeloid
Computing communication score between Myeloid and CAFs
Computing communication score between T-cells and B-cells
Computing communication score between Myeloid and T-cells
Computing communication score between PVL and Plasmablasts
Computing communication score between Pl

In [8]:
# interactions.compute_pairwise_communication_scores() return result: 
interactions.interaction_space.interaction_elements['communication_matrix'].to_csv("./output/communication_matrix.csv")

In [9]:
interactions.compute_pairwise_cci_scores()

Computing pairwise interactions
Computing interaction score between Cancer Epithelial and Cancer Epithelial
Computing interaction score between Endothelial and PVL
Computing interaction score between T-cells and T-cells
Computing interaction score between PVL and T-cells
Computing interaction score between B-cells and Myeloid
Computing interaction score between CAFs and Myeloid
Computing interaction score between Myeloid and T-cells
Computing interaction score between PVL and Plasmablasts
Computing interaction score between Endothelial and Endothelial
Computing interaction score between Plasmablasts and T-cells
Computing interaction score between Endothelial and Myeloid
Computing interaction score between B-cells and CAFs
Computing interaction score between CAFs and Plasmablasts
Computing interaction score between Cancer Epithelial and Endothelial
Computing interaction score between B-cells and B-cells
Computing interaction score between Endothelial and Plasmablasts
Computing interacti

In [10]:
# interactions.compute_pairwise_cci_scores() return result:
interactions.interaction_space.interaction_elements['cci_matrix'].to_csv("./output/cci_matrix.csv")

In [11]:
cci_pvals = interactions.permute_cell_labels(evaluation='interactions', verbose=True, random_state = 123)

100%|██████████| 100/100 [01:14<00:00,  1.34it/s]


In [13]:
cci_pvals.to_csv("./output/cci_pvals.csv")

In [14]:
ccc_pvals = interactions.permute_cell_labels(evaluation='communication',verbose=True, random_state = 123)

100%|██████████| 100/100 [01:19<00:00,  1.25it/s]


In [15]:
ccc_pvals.to_csv("./output/ccc_pval.csv")